# 项目说明
该项目复现local error训练，通过对深度网络的每一层单独计算准确性，最终实现整体的训练。

这是一种新的网络训练方式，可以调控网络每一层的流形，从而实现可解释性的分析。

文献参考：
* [Deep Supervised Learning Using Local Errors](https://www.frontiersin.org/journals/neuroscience/articles/10.3389/fnins.2018.00608/full)
* [Relationship between manifold smoothness and adversarial vulnerability in deep learning with local errors](https://cpb.iphy.ac.cn/EN/10.1088/1674-1056/abd68e)

# 准备

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  数据输入

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 定义数据转换（将图像转换为Tensor并标准化）
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.5,), (0.5,))  # 标准化（均值0.5，标准差0.5）
])

# 下载和加载训练集和测试集
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 使用DataLoader加载数据
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 查看训练数据的一部分
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(images.shape)  # 输出形状，应该是[64, 1, 28, 28]，即64张28x28的图像
print(labels.shape)  # 输出标签形状，应该是[64]


# 网络构建

单层网络和粘合多层网络

In [ ]:
import torch.nn as nn

class SingleLayerNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(SingleLayerNetwork, self).__init__()
        # 定义线性层
        self.fc = nn.Linear(input_size, output_size)
        # 定义ReLU激活函数
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        return x

In [ ]:
import copy

class MultiLayerNetwork(nn.Module):
    def __init__(self):
        super(MultiLayerNetwork, self).__init__()
        self.layers = nn.ModuleList()  # 用于存储逐步添加的网络层

    def add(self, layer):
        # 添加已训练好的网络层到ModuleList中
        self.layers.append(copy.deepcopy(layer))

    def pop(self):
        """删除 self.layers 中的最后一个网络层"""
        if len(self.layers) > 0:
            last_layer = self.layers[-1]  # 获取最后一层
            del self.layers[-1]  # 手动删除
            return last_layer  # 返回被删除的层
        else:
            print("Warning: No layers to remove.")
            return None

    def forward(self, x, n_layers=None, return_intermediate=False):
        outputs = []
        
        # 逐层计算输出
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if return_intermediate and (n_layers is None or i < n_layers):
                outputs.append(x)
            if i == n_layers:
                break
        
        if return_intermediate:
            return outputs
        else:
            return x


# 训练方法
通过读出头训练目标网络

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# 定义读出头网络
class ReadoutHead(nn.Module):
    def __init__(self, input_size, output_size):
        super(ReadoutHead, self).__init__()
        # 初始化权重为高斯分布，且权重不可训练
        self.weight = nn.Parameter(torch.randn(input_size, output_size) * 0.01, requires_grad=False)
        self.bias = nn.Parameter(torch.zeros(output_size), requires_grad=False)

    def forward(self, x):
        # 线性变换：y = xW + b
        return torch.matmul(x, self.weight) + self.bias

In [ ]:
# 定义训练流程
def train_with_readout(fixed_network, target_network, readout_head, data_loader, optimizer, criterion, device):
    if fixed_network is not None:
        fixed_network.eval()    # 固定网络不训练
    target_network.train()      # 目标网络训练
    total_loss = 0

    for inputs, labels in data_loader:
        inputs = inputs.view(inputs.shape[0], -1)  # 将图像展平
        inputs, labels = inputs.to(device), labels.to(device)

        # 如果固定网络不为空，数据先通过固定网络（不计算梯度）
        outputs = inputs
        if fixed_network is not None:
            with torch.no_grad():
                outputs = fixed_network(inputs)


        # 数据通过目标网络
        target_outputs = target_network(outputs)

        # 数据通过读出头网络
        logits = readout_head(target_outputs)

        # 计算交叉熵损失
        loss = criterion(logits, labels)
        total_loss += loss.item()

        # 反向传播优化目标网络
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

In [ ]:
tot_NN = MultiLayerNetwork()
Single_NN = SingleLayerNetwork(28*28, 1000).to(device)
# 定义优化器和损失函数
optimizer = optim.Adam(Single_NN.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
readout_head = ReadoutHead(1000, 10).to(device)

for epoch in range(3):
    loss = train_with_readout(fixed_network=None, target_network=Single_NN, readout_head=readout_head, data_loader=trainloader, optimizer=optimizer, criterion=criterion, device=device)
    print(loss)

tot_NN.add(Single_NN)
tot_NN.add(readout_head)

# 测试正确率

In [ ]:
def evaluate_accuracy(target_network, data_loader, device, readout_head=None):
    # 固定网络、目标网络和读出头网络都设置为评估模式
    target_network.eval()
    if readout_head is not None:
        readout_head.eval()

    correct = 0
    total = 0

    with torch.no_grad():  # 不计算梯度
        for inputs, labels in data_loader:
            inputs = inputs.view(inputs.shape[0], -1)  # 将图像展平
            inputs, labels = inputs.to(device), labels.to(device)

            # 数据通过目标网络
            target_outputs = target_network(inputs)

            if readout_head is not None:
                # 数据通过读出头网络
                target_outputs = readout_head(target_outputs)

            # 预测类别
            _, predicted = torch.max(target_outputs, dim=1)  # 取概率最大的类别

            # 统计正确预测的数量
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 计算并返回准确率
    accuracy = correct / total
    return accuracy

evaluate_accuracy(target_network=tot_NN, readout_head=None, data_loader=testloader, device=device)

# 构建K层的神经网络

In [ ]:
tot_NN = MultiLayerNetwork()
input_size = 28*28
size_range = [1000, 1000, 1000, 1000]
for k, output_size in enumerate(size_range):
    # 初始化一个单层网络
    Single_NN = SingleLayerNetwork(input_size, output_size).to(device)
    optimizer = optim.Adam(Single_NN.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    readout_head = ReadoutHead(output_size, 10).to(device)

    # 训练该单层网络
    for epoch in range(3):
        loss = train_with_readout(fixed_network=tot_NN, target_network=Single_NN, readout_head=readout_head, data_loader=trainloader, optimizer=optimizer, criterion=criterion, device=device)
        print(loss)

    input_size = output_size
    tot_NN.add(Single_NN)

    eva_value = evaluate_accuracy(target_network=tot_NN, readout_head=readout_head, data_loader=testloader, device=device)
    print("evsl", eva_value)

tot_NN.add(readout_head)

final_eval = evaluate_accuracy(target_network=tot_NN, data_loader=testloader, device=device)
print(final_eval)

# 特征指标


## 计算幂率指数$\alpha$

幂律分布的概率密度函数（PDF）一般形式为：

$$P(x) \propto x^{-\alpha}$$

或者更具体地写成：

$$P(x) = C x^{-\alpha}, \quad x \geq x_{\min}$$

其中：
- $x$ 是一个随机变量（如网络节点的度、地震震级、财富分布等）。
- $\alpha$ 是幂律指数（Power-Law Exponent），决定分布的陡峭程度。
- $x_{\min}$ 是幂律分布的最小适用值，在某些数据中，幂律分布可能只适用于某个范围以上的值。
- $C$ 是归一化常数，使得概率总和为 1。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

def estimate_alpha_mle(data, x_min):
    """
    使用最大似然估计（MLE）计算幂律指数 α
    :param data: 观测数据（numpy 数组）
    :param x_min: 设定的最小阈值，幂律分布从 x_min 开始适用
    :return: 估计的 α
    """
    filtered_data = data[data >= x_min]  # 只选取大于等于 x_min 的数据
    n = len(filtered_data)  # 数据点数
    alpha = 1 + n / np.sum(np.log(filtered_data / x_min))
    return alpha

# 生成一个模拟的幂律分布数据
np.random.seed(42)
n_samples = 1000
alpha_true = 2.5  # 真实幂律指数
x_min = 1  # 设定最小阈值

# 生成服从幂律分布的数据（使用逆变换采样法）
random_values = np.random.uniform(size=n_samples)
data = x_min * (1 - random_values) ** (-1 / (alpha_true - 1))

# 估计幂律指数 α
alpha_estimated = estimate_alpha_mle(data, x_min)
print(f"估计的幂律指数 α: {alpha_estimated:.4f}")

# 绘制直方图（对数-对数图）
plt.figure(figsize=(8, 6))
hist, bins, _ = plt.hist(data, bins=500, density=True, alpha=0.6, color='b')
bin_centers = (bins[:-1] + bins[1:]) / 2

# 过滤掉 hist == 0 的数据点，避免 log(0) 错误
valid_indices = hist > 0
log_bin_centers = np.log(bin_centers[valid_indices])
log_hist = np.log(hist[valid_indices])

# 线性回归（拟合幂律指数）
_, intercept, _, _, _ = stats.linregress(log_bin_centers, log_hist)
slope = -alpha_estimated
plt.plot(bin_centers, np.exp(intercept) * bin_centers**slope, 'r--', label=f'fit: slope={slope:.2f}')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('x')
plt.ylabel('P(x)')
plt.title('log-log')
plt.legend()
plt.show()

## 计算决定系数
用于衡量 **主成分保留的方差信息**。

在 **回归分析** 中，$ R^2 $ 衡量模型对数据的解释能力：
$$
R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2}
$$
其中：
- $ y_i $ 是真实值，
- $ \hat{y}_i $ 是模型预测值，
- $ \bar{y} $ 是 $ y $ 的均值。

在 PCA 中，类似的概念是：
$$
R^2 = \frac{\sum_{i=1}^{k} \lambda_i}{\sum_{i=1}^{d} \lambda_i}
$$
其中：
- $ \lambda_i $ 是第 $ i $ 个特征值（主成分的方差）。
- $ k $ 是选取的前 $ k $ 个主成分。
- $ d $ 是所有特征维度（在你的例子中 $ d = 1000 $）。

这个公式表示 **前 $ k $ 个主成分解释了多少比例的总方差**，即 **主成分的累计方差贡献率**。

In [ ]:
import numpy as np

eigenvalues = [3.2, 2.9, 2.5, 1.8, 1.5, 1.2, 0.9, 0.7, 0.5, 0.3]

# 假设 eigenvalues 是特征值数组 (已按降序排列)
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)  # 计算解释方差比例
cumulative_variance = np.cumsum(explained_variance_ratio)  # 计算累计贡献率

# 选择前 k 个主成分，使得累计贡献率达到 95%
k = np.argmax(cumulative_variance >= 0.95) + 1  # 找到累计方差贡献率 >= 95% 的最小维度
R2_95 = cumulative_variance[k - 1]

print(f"To retain 95% variance, we need {k} principal components.")
print(f"R^2 for k={k} components: {R2_95:.4f}")


# 网络信息处理

In [ ]:
import numpy as np
tot_NN.eval()  # 设为评估模式，不启用 Dropout、BatchNorm
output_list = []

with torch.no_grad():  # 不计算梯度，加速推理
    for inputs, labels in trainloader:
        inputs = inputs.view(inputs.shape[0], -1)  # 将图像展平
        inputs, labels = inputs.to(device), labels.to(device)
        output = tot_NN(inputs, 3)  # 前向传播
        output_list.append(output.cpu().numpy())  # 转换为 NumPy 并保存

# 拼接成一个完整的 NumPy 数组
final_output = np.vstack(output_list) 
final_output.shape

In [ ]:
def get_eigenvalues(data):
    data = data - np.mean(data, axis=0)
    # 计算数据的协方差矩阵
    covariance_matrix = np.cov(data, rowvar=False)
    
    # 计算协方差矩阵的特征值
    eigenvalues, _ = np.linalg.eig(covariance_matrix)
    
    # 对特征值进行排序
    eigenvalues = np.sort(eigenvalues)[::-1]
    return eigenvalues

def get_alpha_r(eigenvalues):
    # 估计幂律指数 α
    slope = estimate_alpha_mle(eigenvalues, 0.000001)

    # 计算R^2
    # 假设 eigenvalues 是特征值数组 (已按降序排列)
    explained_variance_ratio = eigenvalues / np.sum(eigenvalues)  # 计算解释方差比例
    cumulative_variance = np.cumsum(explained_variance_ratio)  # 计算累计贡献率

    # 选择前 k 个主成分，使得累计贡献率达到 95%
    k = np.argmax(cumulative_variance >= 0.95) + 1  # 找到累计方差贡献率 >= 95% 的最小维度
    R2_95 = cumulative_variance[10]
    return slope, R2_95, k

eigenvalues = get_eigenvalues(final_output)
slope, R2_95, k = get_alpha_r(eigenvalues)

print(slope)
print(f"To retain 95% variance, we need {k} principal components.")
print(f"R^2 for k={k} components: {R2_95:.4f}")